In [24]:
"import dependencies"
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter


team_Data = pd.read_csv("../Resources/teamData.csv")
player_Data = pd.read_csv("../Resources/playerData.csv")
crossReference_Data = pd.read_csv("../Resources/crossReference.csv")

BEGIN_YEAR = 1986
END_YEAR = 2018


for year in range(BEGIN_YEAR, END_YEAR):
    merged_player_data = player_Data.loc[(player_Data['Year'] >= BEGIN_YEAR) & (player_Data['Year'] < END_YEAR)]

    merged_player_data = merged_player_data[merged_player_data['Year']==year]

    selected_columns = ['Player', 'PER', 'TS%', 'ORB%', 'DRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'WS', '3P%', '2P%', 'FT%', 'PTS', 'MP']
    selected_columns_with_age = ['Player', 'PER', 'TS%', 'ORB%', 'DRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'WS', '3P%', '2P%', 'FT%', 'PTS', 'MP', 'Age']

    subset2py = merged_player_data[selected_columns]
    subset_with_age = merged_player_data[selected_columns_with_age]
    subset2py2 = subset2py

    Q1=subset2py2.quantile(0.25)
    Q3=subset2py2.quantile(0.75)
    IQR=Q3-Q1

    #set up quantiles and IQR with age
    Q1a = subset_with_age.quantile(0.25)
    Q3a = subset_with_age.quantile(0.75)
    IQRa = Q3a-Q1a

    subset2py_no_outlier = subset2py2[~((subset2py2<(Q1-1.5*IQR)))]


    percentile_df = pd.DataFrame()
    for column in selected_columns[1:]:
    #fileter by year
        percentile_df[f"{column}_percentile"] = merged_player_data[column].rank(pct = True)
    
    percentile_df['Player Name'] = merged_player_data['Player']
    percentile_df = percentile_df.dropna().reset_index()

    main_perc_list = []


    for index in range(0,percentile_df.shape[0]):
        row_sum = sum(list(percentile_df.iloc[index])[1:-2])
        row_main_perc = (row_sum/15)
        main_perc_list.append(row_main_perc)

    main_percentile = pd.DataFrame(main_perc_list)
    print('creating main percentile for year ' + str(year))
    percentile_df['Main Percentile'] = main_percentile
    [row, cols] = percentile_df.shape

    def mean(data):
        total = sum(data)
        return total / len(data)


    #numerator : list -> float
    def numerator(data):
        SD_mean = mean(data)
        total = 0 
        for val in data:
            total = total + ((val - SD_mean)**2)
        return total


    #std : list -> float
    def std(data):
        num =  numerator(data)
        den = len(data)
        return math.sqrt(num/den)


    #new_formula : list -> list
    def new_formula(data): 
        percentile = []
        std_fun = std(data)
        mean_fun = mean(data)
        for value in data:
            adjusted_value = (value - mean_fun)/std_fun
            percentile.append(adjusted_value)
        return percentile
    
    adjusted_percentile_df  = pd.DataFrame(new_formula(percentile_df['Main Percentile']))

    print(std(percentile_df['Main Percentile']))
    print(mean(percentile_df['Main Percentile']))

    percentile_df['Adjusted Percentile'] = adjusted_percentile_df

    
    percentile_df.to_csv(f'../Resources/yearly_csvs/{year}_data.csv')


<ipython-input-24-f420c00d7803>:40: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  subset2py_no_outlier = subset2py2[~((subset2py2<(Q1-1.5*IQR)))]
<ipython-input-24-f420c00d7803>:40: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  subset2py_no_outlier = subset2py2[~((subset2py2<(Q1-1.5*IQR)))]
<ipython-input-24-f420c00d7803>:40: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  subset2py_no_outlier = subset2py2[~((subset2py2<(Q1-1.5*IQR)))]
creating main percentile for year 1986
0

In [44]:
resources_path = ('../Resources/yearly_csvs')
yearly_list = []
for item in os.listdir(resources_path): 
    item_path = os.path.join(resources_path, item)
    if os.path.join(resources_path, item): #if it is a file we will continue
        read_csv = pd.read_csv(item_path)
        temp = pd.DataFrame(read_csv)
        yearly_list.append(temp)
        


In [51]:
yearly_list[0]["Adjusted Percentile"]

0      False
1      False
2      False
3      False
4      False
       ...  
290    False
291    False
292    False
293    False
294    False
Name: Adjusted Percentile, Length: 295, dtype: bool